In [1]:
from preprocessing import Audio
import tensorflow as tf
import tf_train as tft
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy as sp
import librosa as li
import librosa.display
import IPython.display as ipd

%matplotlib inline
#tf.debugging.set_log_device_placement(True)

In [2]:
GPU = tf.config.list_physical_devices('GPU')
CPU = tf.config.list_physical_devices('CPU')
DEVICE = GPU[0].name if GPU else CPU[0].name
DEVICE

'/physical_device:GPU:0'

<h1>Подготовка данных для обучения и тестирования</h1>

In [3]:
def get_data():
    def norm(x):
        std = np.std(x ,ddof=1)
        return (x - np.mean(x)) / std

    PATH_0 = r'AudioData/0 Данные' # negative class
    PATH_1 = r'AudioData/1 Джарвис' # positive class
    PATH_2 = r'AudioData/2 Дио' # positive class
    PATH_3 = r'AudioData/3 Итан' # positive class
    PATH_4 = r'AudioData/4 Лада' # positive class
    PATH_5 = r'AudioData/5 Мира' # positive class

    PATHS = li.util.find_files(PATH_0)[:3000] + li.util.find_files(PATH_1) + li.util.find_files(PATH_2) + li.util.find_files(PATH_3) + li.util.find_files(PATH_5) + li.util.find_files(PATH_5)
    N = len(PATHS)

    np.random.shuffle(PATHS)

    SOUNDS = []
    for path in PATHS:
        SOUNDS.append(Audio(path))
        print('Loading {:.1f}%'.format(len(SOUNDS) / N * 100))
        ipd.clear_output(wait=True)
    DATA = []
    LABELS = []

    for sound in SOUNDS:
        DATA.append(norm(sound.e_parts))
        LABELS.append(sound.label)

    return (np.array(DATA), np.array(LABELS))

In [4]:
data, labels = get_data()

Loading 100.0%


In [88]:
test_data, test_labels = data[8000:], labels[8000:]
data, labels = data[:8000], labels[:8000]

In [5]:
data.shape, labels.shape

((3361, 20), (3361,))

In [48]:
BATCH_SIZE = 1081

dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(BATCH_SIZE)
dataset

<BatchDataset shapes: ((None, 20), (None,)), types: (tf.float64, tf.int32)>

In [56]:
BATCH_SIZE = 3361
test_data = np.reshape(data, [3361, 1, 20])
test_labels = np.reshape(labels, [1, 3361])

dataset = tf.data.Dataset.from_tensor_slices((test_data, labels))
dataset = dataset.batch(BATCH_SIZE)
dataset

<BatchDataset shapes: ((None, 1, 20), (None,)), types: (tf.float64, tf.int32)>

In [188]:
data = np.reshape(data, (10070, 20))

In [211]:
labels = np.reshape(labels, (10070,))

In [58]:
test_labels[:100]

array([[0, 0, 0, ..., 0, 0, 0]])

<h1>Создание модели нейронной сети</h1>

In [60]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1, 20,)),
    tf.keras.layers.GRU(20, kernel_regularizer=tf.keras.regularizers.l2(0.01), reset_after=False),
    # tf.keras.layers.Dense(100, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    # tf.keras.layers.Dense(50, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    # tf.keras.layers.Dense(25, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    # tf.keras.layers.Dense(12, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(6)
], name='All_GRU')

model.summary()

Model: "All_GRU"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_4 (GRU)                  (None, 20)                2460      
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 126       
Total params: 2,586
Trainable params: 2,586
Non-trainable params: 0
_________________________________________________________________


PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')

<h1>Обучение</h1>

In [65]:
tft.train(
    train_dataset=dataset,
    model=model,
    epochs=1000,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

Epoch 000 -- Loss(SparseCategoricalCrossentropy): 0.583, Accuracy: 0.8893186450004578
Epoch 010 -- Loss(SparseCategoricalCrossentropy): 0.578, Accuracy: 0.8893186450004578
Epoch 020 -- Loss(SparseCategoricalCrossentropy): 0.573, Accuracy: 0.8893186450004578
Epoch 030 -- Loss(SparseCategoricalCrossentropy): 0.568, Accuracy: 0.8893186450004578
Epoch 040 -- Loss(SparseCategoricalCrossentropy): 0.564, Accuracy: 0.8893404006958008
Epoch 050 -- Loss(SparseCategoricalCrossentropy): 0.559, Accuracy: 0.8893945217132568
Epoch 060 -- Loss(SparseCategoricalCrossentropy): 0.555, Accuracy: 0.889430820941925
Epoch 070 -- Loss(SparseCategoricalCrossentropy): 0.551, Accuracy: 0.8894569277763367
Epoch 080 -- Loss(SparseCategoricalCrossentropy): 0.547, Accuracy: 0.8894765973091125
Epoch 090 -- Loss(SparseCategoricalCrossentropy): 0.543, Accuracy: 0.8894919157028198
Epoch 100 -- Loss(SparseCategoricalCrossentropy): 0.539, Accuracy: 0.8895101547241211
Epoch 110 -- Loss(SparseCategoricalCrossentropy): 0.536

In [36]:
model.save('models\{}-v1.0.h5'.format(model.name))

In [8]:
model = tf.keras.models.load_model(r'models\Mira-v1.0.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                252       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 26        
Total params: 278
Trainable params: 278
Non-trainable params: 0
_________________________________________________________________


In [37]:
with tf.device("CPU"):
    predictions = tft.predict_classes(model=model, features=test_data)

In [38]:
n = 0

for i in range(len(labels)):
    n += int(labels[i] == predictions[i])

print(n, n / len(labels) * 100)

3361 100.0


In [41]:
predictions[60:70], labels[60:70]

(<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 1, 1, 0, 0, 0, 0, 3, 0], dtype=int64)>,
 array([0, 0, 1, 1, 0, 0, 0, 0, 3, 0]))

In [40]:
predictions[:15], labels[:15]

(<tf.Tensor: shape=(15,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int64)>,
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]))

In [30]:
labels[60:70]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [69]:
filename = 'test.tfrecord'
writer_data = tf.data.experimental.TFRecordWriter("datasets\Mira\data.tfrecord")
writer_labels = tf.data.experimental.TFRecordWriter("datasets\Mira\labels.tfrecord")
test_data = tf.data.Dataset.from_tensor_slices(data)
test_labels = tf.data.Dataset.from_tensor_slices(labels)
test_data = test_data.map(lambda x: str(x))
test_labels = test_labels.map(lambda x: str(x))
writer_data.write(test_data)
writer_labels.write(test_labels)

In [75]:
loaded_dataset = tf.data.TFRecordDataset(["datasets\Mira\labels.tfrecord"])

In [76]:
for d in loaded_dataset.take(1):
    print(d)

tf.Tensor(b'Tensor("args_0:0", shape=(), dtype=int32)', shape=(), dtype=string)


In [77]:
loaded_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [83]:
for d in test_data.take(1):
    print(d.numpy())

b'Tensor("args_0:0", shape=(20,), dtype=float64)'


In [74]:
m = tf.keras.metrics.Recall(class_id=1)
m.update_state([1, 0, 1, 2], [1, 0, 1, 2])
m.result().numpy()

InvalidArgumentError: predictions must be <= 1
Condition x <= y did not hold.
First 3 elements of x:
[1. 0. 1.]
First 1 elements of y:
[1.]

In [82]:
y_pred = tf.nn.softmax(model(test_data[:1]))[0]
y_pred

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([0.98039216, 0.00272254, 0.00372326, 0.00742231, 0.00221274,
       0.00352691], dtype=float32)>

In [98]:
m = tf.keras.metrics.TruePositives()
m.update_state([1, 2, 3], [1, 2, 3])
m.result().numpy()

InvalidArgumentError: predictions must be <= 1
Condition x <= y did not hold.
First 3 elements of x:
[1. 2. 3.]
First 1 elements of y:
[1.]

In [94]:
predictions[:100], labels[:100]

(<tf.Tensor: shape=(100,), dtype=int64, numpy=
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 3,
        0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>,
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 3,
        0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0]))